In [1]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available")
    device = torch.device("cuda")
else:
    print("CUDA is not available")
    device = torch.device("cpu")

# Move tensor to the GPU
x = torch.tensor([1.0])
x = x.to(device)
print(x)

CUDA is available
tensor([1.], device='cuda:0')


In [2]:
# we are going to train a simple neural network with an 
# input layer of 71 doubles and middle layer of 101 doubles and an output layer of 71 doubles
# we will use the sigmoid function as the activation function
# we will use the mean squared error as the loss function
# we will use the stochastic gradient descent as the optimization algorithm
# we will use the learning rate of 0.01
# we will use the batch size of 100
# we will use the number of epochs of 100
# we will use the random seed of 42
# will import the training dat from the file "x_training.csv" in the same folder as this notebook
# will import the target data from the file "y_labels.csv" in the same folder as this notebook
# we will save the trained model to the file "model.pt" in the same folder as this notebook
# we will save the loss history to the file "loss_history.csv" in the same folder as this notebook
# we will save the accuracy history to the file "accuracy_history.csv" in the same folder as this notebook
# we will save the training history to the file "training_history.csv" in the same folder as this notebook
# we will save the validation history to the file "validation_history.csv" in the same folder as this notebook
# we will save the test history to the file "test_history.csv" in the same folder as this notebook
# we will save the confusion matrix to the file "confusion_matrix.csv" in the same folder as this notebook

# import the libraries
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import random
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

# set the random seed
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# load the data
x = pd.read_csv("../x_training.csv")
y = pd.read_csv("../y_labels.csv")



In [3]:


# split the data into training, validation and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


In [4]:

# convert the data to tensors
x_train = torch.tensor(x_train.values, dtype=torch.float32)
x_val = torch.tensor(x_val.values, dtype=torch.float32)
x_test = torch.tensor(x_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)


In [5]:

# create the model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(71, 131)
        self.layer2 = nn.Linear(131, 71)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.layer1(x)
        x = self.sigmoid(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x
    


In [6]:
# create the model
model = NeuralNetwork()
model.to(device)

# create the loss function
criterion = nn.MSELoss()

# create the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.05)


In [7]:

# create the data loaders
train_loader = torch.utils.data.DataLoader(list(zip(x_train, y_train)), batch_size=100, shuffle=True)
val_loader = torch.utils.data.DataLoader(list(zip(x_val, y_val)), batch_size=100, shuffle=False)
test_loader = torch.utils.data.DataLoader(list(zip(x_test, y_test)), batch_size=100, shuffle=False)


In [8]:

# train the model
loss_history = []
accuracy_history = []
training_history = []
validation_history = []
for epoch in range(1000):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        predicted = torch.round(outputs)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    loss_history.append(running_loss / len(train_loader))
    accuracy_history.append(correct / total)
    training_history.append(correct / total)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = torch.round(outputs)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    validation_history.append(correct / total)


In [9]:

# save the model
torch.save(model, "model.pt")


In [10]:

# save the loss history
loss_history = pd.DataFrame(loss_history, columns=["loss"])
loss_history.to_csv("loss_history.csv", index=False)

# save the accuracy history
accuracy_history = pd.DataFrame(accuracy_history, columns=["accuracy"])
accuracy_history.to_csv("accuracy_history.csv", index=False)

# save the training history
training_history = pd.DataFrame(training_history, columns=["training"])
training_history.to_csv("training_history.csv", index=False)

# save the validation history
validation_history = pd.DataFrame(validation_history, columns=["validation"])
validation_history.to_csv("validation_history.csv", index=False)


In [13]:
print(torch.cuda.device_count())

1


In [14]:
# make one prediction chosen randomly from the test set.  Comment every line of code
model.eval()
with torch.no_grad():
    i = np.random.randint(0, len(x_test))
    inputs = x_test[i].to(device)
    label = y_test[i].to(device)
    output = model(inputs)
    print("Input:", inputs)
    print("Label:", label)
    print("Output:", output)
    print("Prediction:", torch.round(output))
    print("Actual:", label)
    print("Correct:", torch.round(output) == label)



Input: tensor([0.3000, 0.0000, 0.6000, 0.0000, 0.0000, 0.0000, 1.2000, 0.9000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 1.2000, 0.0000, 0.0000, 0.4000, 0.0000,
        0.6000, 0.6000, 0.0000, 1.1000, 1.2000, 1.0000, 0.2000, 0.0000, 0.0000,
        0.6000, 0.0000, 0.0000, 1.2000, 0.8000, 0.1000, 0.6000, 0.0000, 0.0000,
        0.0000, 1.2000, 0.0000, 0.7000, 0.4000, 0.0000, 0.6000, 0.0000, 0.0000,
        1.1000, 1.2000, 0.0000, 0.5000, 0.6000, 0.0000, 0.0000, 0.0000, 0.0000,
        1.2000, 0.0000, 0.3000, 0.6000, 0.0000, 0.0000, 0.0000, 0.0000, 1.2000,
        0.9000, 0.0000, 0.1000, 0.1000, 0.1000, 0.1000, 0.0000, 2.6000],
       device='cuda:0')
Label: tensor([0.3000, 0.0000, 0.6000, 0.0000, 0.0000, 0.0000, 1.2000, 0.9000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 1.2000, 0.0000, 0.0000, 0.4000, 0.0000,
        0.6000, 0.6000, 0.0000, 1.1000, 1.2000, 1.0000, 0.2000, 0.0000, 0.0000,
        0.6000, 0.0000, 0.0000, 1.2000, 0.8000, 0.1000, 0.0000, 0.0000, 0.6000,
        0

In [ ]:
# 

In [12]:

# test the model
model.eval()
correct = 0
total = 0
predictions = []
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted = torch.round(outputs)
        total += labels.size(0)
        try:
            correct += (predicted == labels).sum().item()
        except Exception as e:
            print("Exception details:")
            # print(e)
        try:    
            predictions.extend(predicted.cpu().numpy())
        # catch the exception object and use it to print details about the exception
        except Exception as e:
            print("Exception details:")
            # print(e)
            # raise the exception again to stop the program
            # raise e
predictions = pd.DataFrame(predictions, columns=["predictions"])
predictions.to_csv("predictions.csv", index=False)

# save the test history
test_history = pd.DataFrame([correct / total], columns=["test"])
test_history.to_csv("test_history.csv", index=False)


ValueError: 1 columns passed, passed data had 71 columns

In [ ]:

# save the confusion matrix
confusion_matrix = confusion_matrix(y_test, predictions)
confusion_matrix = pd.DataFrame(confusion_matrix)
confusion_matrix.to_csv("confusion_matrix.csv", index=False)

# plot the loss history
plt.plot(loss_history)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss History")
plt.show()

# plot the accuracy history
plt.plot(accuracy_history)
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy History")
plt.show()

# plot the training and validation history
plt.plot(training_history, label="Training")
plt.plot(validation_history, label="Validation")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training and Validation History")
plt.legend()
plt.show()

# print the test history
print(test_history)

# print the confusion matrix
print(confusion_matrix)

# print the predictions
print(predictions)

# print the model
print(model)

# print the loss history
print(loss_history)

# print the accuracy history
print(accuracy_history)

# print the training history
print(training_history)

# print the validation history
print(validation_history)


